In [1]:
!git clone https://github.com/LeC-Z/RWKV-nonogram.git
!pip install rwkv
%cd RWKV-nonogram/
!mv ./rwkv_vocab_nonogram.txt rwkv_vocab_v20230424.txt
!cp ./rwkv_vocab_v20230424.txt /usr/local/lib/python3.10/dist-packages/rwkv/

Cloning into 'RWKV-nonogram'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 23 (delta 8), reused 5 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (23/23), 38.79 MiB | 8.87 MiB/s, done.
Resolving deltas: 100% (8/8), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 406.2/406.2 kB 14.1 MB/s eta 0:00:00


In [24]:
from rwkv.model import RWKV
from rwkv.utils import PIPELINE, PIPELINE_ARGS

# download models: https://huggingface.co/BlinkDL
model = RWKV(model='./rwkv-11.pth', strategy='cpu fp32')
pipeline = PIPELINE(model, "rwkv_vocab_v20230424") # 20B_tokenizer.json is in https://github.com/BlinkDL/ChatRWKV
# use pipeline = PIPELINE(model, "rwkv_vocab_v20230424") for rwkv "world" models

def my_print(s):
    print(s, end='', flush=True)


RWKV_JIT_ON 1 RWKV_CUDA_ON 0 RESCALE_LAYER 0

Loading ./rwkv-11.pth ...


/usr/local/lib/python3.10/dist-packages/rwkv/model.py:206: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.w = torch.load(args.MODEL_NAME, map_location='cpu') # load mode

Model detected: v6.0
Strategy: (total 12+1=13 layers)
* cpu [float32, float32], store 13 layers
0-cpu-float32-float32 1-cpu-float32-float32 2-cpu-float32-float32 3-cpu-float32-float32 4-cpu-float32-float32 5-cpu-float32-float32 6-cpu-float32-float32 7-cpu-float32-float32 8-cpu-float32-float32 9-cpu-float32-float32 10-cpu-float32-float32 11-cpu-float32-float32 12-cpu-float32-float32 
emb.weight                        f32      cpu     17   384       
blocks.0.ln1.weight               f32      cpu    384             
blocks.0.ln1.bias                 f32      cpu    384             
blocks.0.ln2.weight               f32      cpu    384             
blocks.0.ln2.bias                 f32      cpu    384             
blocks.0.att.time_maa_x           f32      cpu    384             
blocks.0.att.time_maa_w           f32      cpu    384             
blocks.0.att.time_maa_k           f32      cpu    384             
blocks.0.att.time_maa_v           f32      cpu    384             
blocks.0.at

using the similar format as follow **exactly**:
> "[[3], [2], [4], [2], [2]]\n[[3], [4], [3], [1, 1], [1]]\n\n"

[3], [2], [4], [2], [2]] is the row prompts.

[[3], [4], [3], [1, 1], [1]] is the col prompts.


`']]'` is the symbols of end.

In [25]:
args = PIPELINE_ARGS(temperature = 0.8, top_p = 0.2, top_k = 100, # top_k = 0 then ignore
                     alpha_frequency = 0,
                     alpha_presence = 0,
                     alpha_decay = 0.996, # gradually decay the penalty
                     token_ban = [0], # ban the generation of some tokens
                     token_stop = [''], # stop generation whenever you see any token here
                     chunk_len = 1024) # split input into chunks to save VRAM (shorter -> slower)


ctx = "[[3], [2], [4], [2], [2]]\n[[3], [4], [3], [1, 1], [1]]\n\n"
pipeline.generate(ctx, token_count=3000, args=args, callback=my_print)

[3] [4] [3] [1, 1] [1] 
00000:[3]
00000:[2]
00000:[4]
00000:[2]
00000:[2]
R1:
[3] [4] [3] [1, 1] [1] 
00#00:[3]
00000:[2]
00000:[4]
00000:[2]
00000:[2]
R2:
[3] [4] [3] [1, 1] [1] 
00#00:[3]
00000:[2]
00000:[4]
00000:[2]
00000:[2]
R3:
[3] [4] [3] [1, 1] [1] 
00#00:[3]
00000:[2]
0###0:[4]
00000:[2]
00000:[2]
R4:
[3] [4] [3] [1, 1] [1] 
00#00:[3]
00000:[2]
0###0:[4]
00000:[2]
00000:[2]
R5:
[3] [4] [3] [1, 1] [1] 
00#00:[3]
00000:[2]
0###0:[4]
00000:[2]
00000:[2]
C1:
[3] [4] [3] [1, 1] [1] 
00#00:[3]
00000:[2]
####0:[4]
00000:[2]
00000:[2]
C2:
[3] [4] [3] [1, 1] [1] 
00#00:[3]
0#000:[2]
####0:[4]
0#000:[2]
00000:[2]
C3:
[3] [4] [3] [1, 1] [1] 
00#00:[3]
0##00:[2]
####0:[4]
0#X00:[2]
00X00:[2]
C4:
[3] [4] [3] [1, 1] [1] 
00#00:[3]
0##X0:[2]
####0:[4]
0#XX0:[2]
00X00:[2]
C5:
[3] [4] [3] [1, 1] [1] 
00#00:[3]
0##X0:[2]
####0:[4]
0#XX0:[2]
00X00:[2]
R1:
[3] [4] [3] [1, 1] [1] 
00#00:[3]
0##X0:[2]
####0:[4]
0#XX0:[2]
00X00:[2]
R2:
[3] [4] [3] [1, 1] [1] 
00#00:[3]
X##XX:[2]
####0:[4]
0#XX0:[2]


'[3] [4] [3] [1, 1] [1] \n00000:[3]\n00000:[2]\n00000:[4]\n00000:[2]\n00000:[2]\nR1:\n[3] [4] [3] [1, 1] [1] \n00#00:[3]\n00000:[2]\n00000:[4]\n00000:[2]\n00000:[2]\nR2:\n[3] [4] [3] [1, 1] [1] \n00#00:[3]\n00000:[2]\n00000:[4]\n00000:[2]\n00000:[2]\nR3:\n[3] [4] [3] [1, 1] [1] \n00#00:[3]\n00000:[2]\n0###0:[4]\n00000:[2]\n00000:[2]\nR4:\n[3] [4] [3] [1, 1] [1] \n00#00:[3]\n00000:[2]\n0###0:[4]\n00000:[2]\n00000:[2]\nR5:\n[3] [4] [3] [1, 1] [1] \n00#00:[3]\n00000:[2]\n0###0:[4]\n00000:[2]\n00000:[2]\nC1:\n[3] [4] [3] [1, 1] [1] \n00#00:[3]\n00000:[2]\n####0:[4]\n00000:[2]\n00000:[2]\nC2:\n[3] [4] [3] [1, 1] [1] \n00#00:[3]\n0#000:[2]\n####0:[4]\n0#000:[2]\n00000:[2]\nC3:\n[3] [4] [3] [1, 1] [1] \n00#00:[3]\n0##00:[2]\n####0:[4]\n0#X00:[2]\n00X00:[2]\nC4:\n[3] [4] [3] [1, 1] [1] \n00#00:[3]\n0##X0:[2]\n####0:[4]\n0#XX0:[2]\n00X00:[2]\nC5:\n[3] [4] [3] [1, 1] [1] \n00#00:[3]\n0##X0:[2]\n####0:[4]\n0#XX0:[2]\n00X00:[2]\nR1:\n[3] [4] [3] [1, 1] [1] \n00#00:[3]\n0##X0:[2]\n####0:[4]\n0#XX0:

In [4]:
!dir

LICENSE  README.md  rwkv-11.pth  rwkv.gif  rwkv_vocab_nonogram.txt  visualize.gif
